In [1]:
import os
import pdfplumber

In [2]:
combined_text = ''

In [ ]:
files_directory = 'files'

# Loop through all the files in the directory
for filename in os.listdir(files_directory):
    if filename.endswith('.pdf'):
        # Open the pdf file
        with pdfplumber.open(os.path.join(files_directory, filename)) as pdf:
            # Loop through all pages in the pdf file
            for page in pdf.pages:
                # Extract teh text from the page and add it to the rest of the text
                combined_text += page.extract_text()+ ' '

print(combined_text)


IndentationError: expected an indented block after 'if' statement on line 5 (4259697203.py, line 7)